In [18]:
import torch
import os
import cv2
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [19]:
from torchvision import transforms

In [20]:
transform_data = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(),
        transforms.RandomHorizontalFlip(),
        transforms.RandomCrop((112)),
        transforms.ToTensor(),
    ]
)

In [21]:
def load_data(img_size=112):
    data = []
    labels = {}
    index = -1
    for label in os.listdir('./data/'):
        index += 1
        labels[label] = index
    print(len(labels))
    X = []
    y = []
    for label in labels:
        for file in os.listdir(f'./data/{label}/'):
            path = f'./data/{label}/{file}'
            img = cv2.imread(path)
            img = cv2.resize(img,(img_size,img_size))
            data.append([np.array(transform_data(np.array(img))),labels[label]])
            X.append(np.array(transform_data(np.array(img))))
            y.append(labels[label])
    np.random.shuffle(data)
    np.random.shuffle(data)
    np.random.shuffle(data)
    np.random.shuffle(data)
    np.random.shuffle(data)
    np.save('./data.npy',data)
    VAL_SPLIT = 0.25
    VAL_SPLIT = len(X)*VAL_SPLIT
    VAL_SPLIT = int(VAL_SPLIT)
    X_train = X[:-VAL_SPLIT]
    y_train = y[:-VAL_SPLIT]
    X_test = X[-VAL_SPLIT:]
    y_test = y[-VAL_SPLIT:]
    X = torch.from_numpy(np.array(X))
    y = torch.from_numpy(np.array(y))
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    X_train = torch.from_numpy(X_train)
    X_test = torch.from_numpy(X_test)
    y_train = torch.from_numpy(y_train)
    y_test = torch.from_numpy(y_test)
    return X,y,X_train,X_test,y_train,y_test

In [22]:
X,y,X_train,X_test,y_train,y_test = load_data()

50


/home/indika/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


## Modelling

In [23]:
import torch.nn as nn
import torch.nn.functional as F

In [24]:
class BaseLine(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,32,5)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv2batchnorm = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64,128,5)
        self.fc1 = nn.Linear(128*10*10,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,50)
        self.relu = nn.ReLU()
    
    def forward(self,X):
        preds = F.max_pool2d(self.relu(self.conv1(X)),(2,2))
        preds = F.max_pool2d(self.relu(self.conv2batchnorm(self.conv2(preds))),(2,2))
        preds = F.max_pool2d(self.relu(self.conv3(preds)),(2,2))
        preds = preds.view(-1,128*10*10)
        preds = self.relu(self.fc1(preds))
        preds = self.relu(self.fc2(preds))
        preds = self.relu(self.fc3(preds))
        return preds

In [25]:
device = torch.device('cuda')

In [26]:
from torchvision import models

In [27]:
# model = BaseLine().to(device)
# model = model.to(device)

model = models.resnet18(pretrained=True).to(device)
in_f = model.fc.in_features
model.fc = nn.Linear(in_f,50)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)

In [28]:
PROJECT_NAME = 'Car-Brands-Images-Clf'

In [29]:
import wandb

In [30]:
EPOCHS = 100
BATCH_SIZE = 32

In [31]:
from tqdm import tqdm

In [32]:
def get_loss(criterion,y,model,X):
    model.to('cuda')
    preds = model(X.view(-1,3,112,112).to('cuda').float())
    preds.to('cuda')
    loss = criterion(preds,torch.tensor(y,dtype=torch.long).to('cuda'))
    loss.backward()
    return loss.item()
def test(net,X,y):
    device = 'cuda'
    net.to(device)
    correct = 0
    total = 0
    net.eval()
    with torch.no_grad():
        for i in range(len(X)):
            real_class = torch.argmax(y[i]).to(device)
            net_out = net(X[i].view(-1,3,112,112).to(device).float())
            net_out = net_out[0]
            predictied_class = torch.argmax(net_out)
            if predictied_class == real_class:
                correct += 1
            total += 1
    net.train()
    net.to('cuda')
    return round(correct/total,3)

In [34]:
# wandb.init(project=PROJECT_NAME,name='transfer-learning')
# for _ in tqdm(range(EPOCHS)):
#     for i in range(0,len(X_train),BATCH_SIZE):
#         X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
#         y_batch = y_train[i:i+BATCH_SIZE].to(device)
#         model.to(device)
#         preds = model(X_batch)
#         preds = preds.to(device)
#         loss = criterion(preds,y_batch)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,y_test,model,X_test),'accuracy':test(model,X_train,y_train),'val_accuracy':test(model,X_test,y_test)})

In [35]:
# TL vs Custom Model best = TL

In [36]:
EPOCHS = 100
BATCH_SIZE = 32

In [37]:
model = models.mobilenet_v3_large(pretrained=False, num_classes=50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
wandb.init(project=PROJECT_NAME,name=f'models.mobilenet_v3_large')
for _ in tqdm(range(EPOCHS),leave=False):
    for i in tqdm(range(0,len(X_train),BATCH_SIZE),leave=False):
        X_batch = X_train[i:i+BATCH_SIZE].view(-1,3,112,112).to(device)
        y_batch = y_train[i:i+BATCH_SIZE].to(device)
        model.to(device)
        preds = model(X_batch)
        preds = preds.to(device)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,y_test,model,X_test),'accuracy':test(model,X_train,y_train),'val_accuracy':test(model,X_test,y_test)})

loss,0.52878
val_loss,8.33492
accuracy,0.0
val_accuracy,0.0
_runtime,1047
_timestamp,1621664983
_step,94


loss,▃▄▁▁█▂▁▁▁▆▆▁▁▁▂▁▅▁▁▂▁▄▂▃▁▁▂▁▅▁▂▁▁▃▃▅▂▂▁▁
val_loss,█▆█▄▂▄▄▃▂▂▂▂▂▂▁▁▂▂▂▂▂▁▁▁▂▂▁▂▂▂▁▂▂▂▁▁▁▁▁▁
accuracy,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_runtime,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_timestamp,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


  0%|          | 0/108 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 104.00 MiB (GPU 0; 11.76 GiB total capacity; 9.21 GiB already allocated; 112.88 MiB free; 9.41 GiB reserved in total by PyTorch)